In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Required Library And Datasets

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import auc,classification_report ,precision_score , recall_score ,accuracy_score, confusion_matrix , roc_curve , RocCurveDisplay
from sklearn.pipeline import Pipeline

# OverView of The Datatsets

In [ ]:
df=pd.read_csv("/kaggle/input/us-adult-income-update/census.csv")
train=pd.read_csv("/kaggle/input/us-adult-income-update/train.csv")
test=pd.read_csv("/kaggle/input/us-adult-income-update/test.csv")

In [ ]:
df.shape , train.shape , test.shape

In [ ]:
32561/48842 , 16281/48842

In [ ]:
df

In [ ]:
#train=train.sample(4000)

In [ ]:
#test=test.sample(1000)

In [ ]:
df.duplicated().sum()

In [ ]:
df=df.drop_duplicates()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df.corr()

In [ ]:
# Numeric Column

In [ ]:
num=df.select_dtypes(exclude="object")
num

In [ ]:
cat=df.select_dtypes(include="object")

In [ ]:
cat

In [ ]:
df["occupation"]=df["occupation"].apply(lambda x: "unknown" if x=="?" else x)

In [ ]:
cat["occupation"]=cat["occupation"].apply(lambda x: "unknown" if x=="?" else x)

# Some Data Analysis

In [ ]:
df.shape

In [ ]:
df.head(1)

In [ ]:
# Distrubution Of Age Is almost Normally distributed

In [ ]:
sns.kdeplot(num["age"])

In [ ]:
plt.figure(figsize=(16,32))
pltnumber=1
for i in num.columns:
    if pltnumber<=6:
        ax=plt.subplot(3,2,pltnumber)
        sns.distplot(num[i])
        plt.xlabel(i , fontsize=20)
    pltnumber=pltnumber+1

In [ ]:
plt.figure(figsize=(16,32))
pltnumber=1
for i in num.columns:
    if pltnumber<=6:
        ax=plt.subplot(3,2,pltnumber)
        sns.boxplot(num[i])
        plt.xlabel(i , fontsize=20)
    pltnumber=pltnumber+1

In [ ]:
#let see the relation between Age and  income
# As we cann see that there is relationship between Age and income

In [ ]:
sns.barplot(df["income"] , df["age"])

In [ ]:
df.head(1)

In [ ]:
num.columns

In [ ]:
plt.figure(figsize=(12,16))
pltnumber=1
for i in num.columns:
    if pltnumber<=6:
        ax=plt.subplot(3,2,pltnumber)
        sns.barplot(df["income"] , df[i])
    pltnumber=pltnumber+1

In [ ]:
for i in num.columns:
    sns.barplot(df["income"] ,df["age"])

In [ ]:
# here We can see there is question Mark in workclass column

In [ ]:
test["workclass"].value_counts()

In [ ]:
df["workclass"]=df["workclass"].replace({"?":"Unknown"})

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(df["income"] , hue=df["workclass"])
plt.show()

In [ ]:
# lets check a replationship between income and fnlwgt
# as we can see that there is no co relationship between this
sns.barplot(df["income"] , df["fnlwgt"])
plt.show()

In [ ]:
# lets check a replationship between income and education-num
# as we can see that there is high relationship between this
sns.barplot(df["income"] , df["education-num"])
plt.show()

In [ ]:
# let see a relation ship between income and education
# as we can see that only master and doctarate people have salary more than 50k
plt.figure(figsize=(12,8))
sns.countplot(df["education"] , hue=df["income"])
plt.xticks(rotation=90)
plt.show()

In [ ]:
# let see a relation ship between income and marital status
# as we can see that only married-civ-spouse can have chances to go more than 50k
plt.figure(figsize=(12,8))
sns.countplot(df["marital-status"] , hue=df["income"])
plt.xticks(rotation=90)
plt.show()

In [ ]:
# let see a relation ship between income and occupation
# as we can see that only amed forces and exes managical people have salary more than 50k
plt.figure(figsize=(12,8))
sns.countplot(df["occupation"] , hue=df["income"])
plt.xticks(rotation=90)
plt.show()

In [ ]:
# let see a relation ship between income and relationalship
# as we can see that only amed forces and exes managical people have salary more than 50k
plt.figure(figsize=(12,8))
sns.countplot(df["relationship"] , hue=df["income"])
plt.xticks(rotation=90)
plt.show()

# Now Model Building

In [ ]:
# here We can Show Datasets Is imbalance Datasets

In [ ]:
df["income"].value_counts()/df.shape[0]

In [ ]:
x=df.drop("income" , axis=1)
y=df["income"]

In [ ]:
y=y.apply(lambda x: 1 if x==">50K" else 0)

In [ ]:
x.shape , y.shape

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rs=RandomUnderSampler(random_state=42)
x,y=rs.fit_resample(x,y)

In [ ]:
x.shape , y.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier

In [ ]:
svc=SVC(kernel="sigmoid" , gamma=1.0)
dtc=DecisionTreeClassifier(max_depth=5)
rfc=RandomForestClassifier(n_estimators=50 , random_state=2)
knc=KNeighborsClassifier()
lrc=LogisticRegression(solver="liblinear" , penalty="l1")
bc=BaggingClassifier(n_estimators=50 , random_state=2)
abc=AdaBoostClassifier(n_estimators=50 , random_state=2)
gbc=GradientBoostingClassifier(n_estimators=50 , random_state=2)
xgb=XGBClassifier(n_estimators=50 ,random_state=2)

In [ ]:
clf=[svc , dtc , rfc , knc , lrc , bc , abc , gbc , xgb]

In [ ]:
algorithm=["SVC" , "DecisionTreeClassifer" , "RandomForestClassifier" , "KnearestNeighbor" , "LogisticRegression" , "BaggingClassifer" , "AdaBoostClassifer" , "GradientBoostClassifer" , "XGboost"]

In [ ]:
step1=ColumnTransformer(transformers=[
    ("col_tnf",OneHotEncoder(sparse=False , drop="first" , handle_unknown="ignore") , [1,3,5,6,7,8,9,13])
    
    
]                
                 ,remainder="passthrough")

In [ ]:
step2=StandardScaler()

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x , y ,test_size=0.3 )

In [ ]:
df.shape , test.shape , test.shape[0]/df.shape[0]

In [ ]:
Accuarcy_score=[] 

In [ ]:
train_acc=[]

In [ ]:
for i in clf:
    pipe=Pipeline([    
        ("step1",step1),
        ("step2",step2),
        ("step3",i)
    ])
    pipe.fit(x_train,y_train)
    y_pred=pipe.predict(x_test)
    acc=accuracy_score(y_pred , y_test)
    Accuarcy_score.append(acc)
    y_train_pred=pipe.predict(x_train)
    train_acc.append(accuracy_score(y_train , y_train_pred))

In [ ]:
final=pd.DataFrame({"Algorithm":algorithm , "Accuracy_Score":Accuarcy_score , "Training_acc":train_acc}).sort_index(ascending=False)
final

# Model Overfitting occure

In [ ]:
df

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x , y ,test_size=0.2 , random_state=2 )

In [ ]:
accuracy2=[]
train_acc2=[]

In [ ]:
for i in clf:
    pipe=Pipeline([    
        ("step1",step1),
        ("step2",step2),
        ("step3",i)
    ])
    pipe.fit(x_train,y_train)
    y_pred=pipe.predict(x_test)
    acc=accuracy_score(y_pred , y_test)
    accuracy2.append(acc)
    y_train_pred=pipe.predict(x_train)
    train_acc2.append(accuracy_score(y_train , y_train_pred))

In [ ]:
final=pd.DataFrame({"Algorithm":algorithm , "Accuracy_Score":accuracy2 , "Training_acc":train_acc2}).sort_index(ascending=False)
final

**applying PCA**

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2)

In [ ]:
accuracy3=[]
train_acc3=[]

In [ ]:
for i in clf:
    pipe=Pipeline([    
        ("step1",step1),
        ("step2",step2),
        ("stpe3" ,pca),
        ("step4",i)
    ])
    pipe.fit(x_train,y_train)
    y_pred=pipe.predict(x_test)
    acc=accuracy_score(y_pred , y_test)
    accuracy3.append(acc)
    y_train_pred=pipe.predict(x_train)
    train_acc3.append(accuracy_score(y_train , y_train_pred))

In [ ]:
final=pd.DataFrame({"Algorithm":algorithm , "Accuracy_Score":accuracy3 , "Training_acc":train_acc3}).sort_index(ascending=False)
final

In [ ]:
bc=BaggingClassifier(n_estimators=100 , random_state=2)
rfc=RandomForestClassifier(n_estimators=50 , random_state=2)

In [ ]:
a=[i for i in range(10,110,10)]

In [ ]:
svc

In [ ]:
bc=BaggingClassifier(n_estimators=100 , random_state=2 , base_estimator=svc)

In [ ]:
for i in a:
    bc=BaggingClassifier(n_estimators=i , random_state=2)
    pipe=Pipeline([    
        ("step1",step1),
        ("step2",step2),
        ("step4",bc)
    ])
    pipe.fit(x_train,y_train)
    print("for" , i)
    y_pred=pipe.predict(x_test)
    print("test_acc",accuracy_score(y_pred , y_test))

    y_train_pred=pipe.predict(x_train)
    print("train_acc",accuracy_score(y_train , y_train_pred))
    print("-"*50)

In [ ]:
bc=BaggingClassifier(n_estimators=50 , random_state=2 , base_estimator=gbc)
pipe=Pipeline([    
    ("step1",step1),
    ("step2",step2),
    ("step4",bc)
])
pipe.fit(x_train,y_train)
y_pred=pipe.predict(x_test)
print("test_acc",accuracy_score(y_pred , y_test))

y_train_pred=pipe.predict(x_train)
print("train_acc",accuracy_score(y_train , y_train_pred))
print("-"*50)

In [ ]:
bc=BaggingClassifier(n_estimators=50 ,max_samples=0.7 , max_features=0.7 , random_state=2 , base_estimator=xgb )
pipe=Pipeline([    
    ("step1",step1),
    ("step2",step2),
    ("step4",bc)
])
pipe.fit(x_train,y_train)
y_pred=pipe.predict(x_test)
print("test_acc",accuracy_score(y_pred , y_test))

y_train_pred=pipe.predict(x_train)
print("train_acc",accuracy_score(y_train , y_train_pred))
print("-"*50)

**Hyper Parametere Tuning**

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris  # Import your dataset here
from sklearn.model_selection import train_test_split

# Load your dataset and split it into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the transformers for different columns
column_transformer = ColumnTransformer(
    transformers=[
        ("col_tnf", OneHotEncoder(sparse=False, drop="first", handle_unknown="ignore"), [1, 3, 5, 6, 7, 8, 9, 13])
    ],
    remainder="passthrough"
)

# Create a Pipeline with the preprocessing steps and the BaggingClassifier
pipeline = Pipeline([
    ('step1', column_transformer),
    ('step2', StandardScaler()),
    ('step3', BaggingClassifier())
])

# Define the hyperparameters and their ranges to search over
param_grid = {
    'step3__n_estimators': [10, 50, 100],
    'step3__max_samples': [0.5, 0.7, 1.0],
    'step3__max_features': [0.5, 0.7, 1.0],
    'step3__bootstrap': [True, False],
    'step3__bootstrap_features': [True, False],
    'step3__random_state': [42],
    # Add any other hyperparameters for your BaggingClassifier here
}

# Create a GridSearchCV instance
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5)

# Fit the GridSearchCV to your data
grid_search.fit(x_train, y_train)

# Print the best hyperparameters found by GridSearchCV
print("Best Hyperparameters: ", grid_search.best_params_)

# Get the best estimator (Pipeline with the best hyperparameters)
best_pipeline = grid_search.best_estimator_

# Evaluate the best Pipeline on your test data



In [ ]:
accuracy = best_pipeline.score(x_test, y_test)
print("Accuracy on Test Data: ", accuracy)

**New Model With Params**

In [ ]:
{'step3__bootstrap': True, 'step3__bootstrap_features': True, 'step3__max_features': 0.7, 'step3__max_samples': 0.7, 'step3__n_estimators': 100, 'step3__random_state': 42}

In [ ]:
bc=BaggingClassifier(bootstrap=True ,bootstrap_features=True ,  n_estimators=100 ,max_samples=0.7 , max_features=0.7 , random_state=2 , base_estimator=xgb )
pipe=Pipeline([    
    ("step1",step1),
    ("step2",step2),
    ("step4",bc)
])
pipe.fit(x_train,y_train)
y_pred=pipe.predict(x_test)
print("test_acc",accuracy_score(y_pred , y_test))

y_train_pred=pipe.predict(x_train)
print("train_acc",accuracy_score(y_train , y_train_pred))
print("-"*50)

In [ ]:
x_test.head(1)

In [ ]:
pipe.predict([[26 ,"Private", 343506 , "HS-grad" ,9 ,"Never-married" ,"Other-service" ,"Own-child" ,"Black" , "Female" ,0 ,0 ,40 ,"United-States"]])

In [ ]:
import pickle

In [ ]:
with open("census_income_predictor.pkl" , "wb") as f:
    pickle.dump(pipe , f)

In [ ]:
with open("census_income_predictor" , "wb") as f:
    pickle.dump(pipe , f)